# Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [38]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fukue,JP,32.6881,128.8419,63.57,78,94,4.54,overcast clouds
1,1,Ancud,CL,-41.8697,-73.8203,52.84,84,94,19.33,light rain
2,2,Albany,US,42.6001,-73.9662,79.74,61,97,7.00,overcast clouds
3,3,Portland,US,45.5234,-122.6762,57.99,66,99,1.99,overcast clouds
4,4,Kaitangata,NZ,-46.2817,169.8464,44.83,82,36,18.52,scattered clouds


In [40]:
# 2. Prompt the user75 to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Albany,US,42.6001,-73.9662,79.74,61,97,7.00,overcast clouds
12,12,Cayenne,GF,4.9333,-52.3333,75.24,1,75,5.75,moderate rain
14,14,Kapaa,US,22.0752,-159.3190,79.12,84,50,1.01,scattered clouds
15,15,Butaritari,KI,3.0707,172.7902,81.46,73,20,16.28,few clouds
21,21,Hilo,US,19.7297,-155.0900,81.63,85,18,8.01,few clouds
23,23,San Policarpo,PH,12.1791,125.5072,81.16,80,100,8.75,overcast clouds
25,25,Nicoya,CR,10.1483,-85.4520,82.29,74,97,6.06,overcast clouds
26,26,Houma,US,29.5958,-90.7195,89.60,64,40,14.97,scattered clouds
35,35,Labrea,BR,-7.2586,-64.7981,86.59,42,40,1.39,scattered clouds
40,40,Bambous Virieux,MU,-20.3428,57.7575,76.21,88,75,5.75,broken clouds


In [42]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Albany,US,79.74,overcast clouds,42.6001,-73.9662,
12,Cayenne,GF,75.24,moderate rain,4.9333,-52.3333,
14,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,
15,Butaritari,KI,81.46,few clouds,3.0707,172.7902,
21,Hilo,US,81.63,few clouds,19.7297,-155.0900,
23,San Policarpo,PH,81.16,overcast clouds,12.1791,125.5072,
25,Nicoya,CR,82.29,overcast clouds,10.1483,-85.4520,
26,Houma,US,89.60,scattered clouds,29.5958,-90.7195,
35,Labrea,BR,86.59,scattered clouds,-7.2586,-64.7981,
40,Bambous Virieux,MU,76.21,broken clouds,-20.3428,57.7575,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Albany,US,79.74,overcast clouds,42.6001,-73.9662,
12,Cayenne,GF,75.24,moderate rain,4.9333,-52.3333,Hôtel Le Dronmi
14,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Butaritari,KI,81.46,few clouds,3.0707,172.7902,Isles Sunset Lodge
21,Hilo,US,81.63,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,San Policarpo,PH,81.16,overcast clouds,12.1791,125.5072,Agapita M. Nicart
25,Nicoya,CR,82.29,overcast clouds,10.1483,-85.4520,Las Cabinas Del Sueño
26,Houma,US,89.60,scattered clouds,29.5958,-90.7195,Courtyard by Marriott Houma
35,Labrea,BR,86.59,scattered clouds,-7.2586,-64.7981,Hotel Novo Horizonte
40,Bambous Virieux,MU,76.21,broken clouds,-20.3428,57.7575,Casa Tia Villa


In [47]:
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [56]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

# Replace empty values with NaN
hotel_df.loc[(hotel_df["Hotel Name"] == ""), ["Hotel Name"]] = np.nan

# Drop NaN rows
clean_hotel_df = hotel_df.dropna()

In [57]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Cayenne,GF,75.24,moderate rain,4.9333,-52.3333,Hôtel Le Dronmi
14,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Butaritari,KI,81.46,few clouds,3.0707,172.7902,Isles Sunset Lodge
21,Hilo,US,81.63,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,San Policarpo,PH,81.16,overcast clouds,12.1791,125.5072,Agapita M. Nicart
25,Nicoya,CR,82.29,overcast clouds,10.1483,-85.4520,Las Cabinas Del Sueño
26,Houma,US,89.60,scattered clouds,29.5958,-90.7195,Courtyard by Marriott Houma
35,Labrea,BR,86.59,scattered clouds,-7.2586,-64.7981,Hotel Novo Horizonte
40,Bambous Virieux,MU,76.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
43,Kirakira,SB,78.85,few clouds,-10.4544,161.9205,SanBiz Lodge


In [58]:
clean_hotel_df.count()

City                   197
Country                197
Max Temp               197
Current Description    197
Lat                    197
Lng                    197
Hotel Name             197
dtype: int64

In [23]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [24]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Albany,US,79.74,overcast clouds,42.6001,-73.9662,
12,Cayenne,GF,75.24,moderate rain,4.9333,-52.3333,Hôtel Le Dronmi
14,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Butaritari,KI,81.46,few clouds,3.0707,172.7902,Isles Sunset Lodge
21,Hilo,US,81.63,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,San Policarpo,PH,81.16,overcast clouds,12.1791,125.5072,Agapita M. Nicart
25,Nicoya,CR,82.29,overcast clouds,10.1483,-85.4520,Las Cabinas Del Sueño
26,Houma,US,89.60,scattered clouds,29.5958,-90.7195,Courtyard by Marriott Houma
35,Labrea,BR,86.59,scattered clouds,-7.2586,-64.7981,Hotel Novo Horizonte
40,Bambous Virieux,MU,76.21,broken clouds,-20.3428,57.7575,Casa Tia Villa


In [26]:
# Figure out why dataframe still includes empty value
hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [59]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
